# Background removal

In this notebook we show how you can use the depth maps to remove most of the background from the images. 
This is important because all data was recorded with the same background, but the models have to be able to generalize to different backgrounds.
The background at the ICRA 2024 will also be different than during the data collection.


In [ ]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from airo_camera_toolkit.point_clouds.conversions import point_cloud_to_open3d
from cloth_tools.dataset.format import load_competition_observation

data_dir = Path("data")
dataset_dir = data_dir / "cloth_competition_dataset_0000"

In [ ]:
os.path.exists(dataset_dir)

In [ ]:
observation_start_dir = dataset_dir / "sample_000000" / "observation_start"

observation = load_competition_observation(observation_start_dir)

In [ ]:
plt.imshow(observation.depth_map)

In [ ]:
distance_max = 1.55
plt.imshow(observation.depth_map < distance_max)

In [ ]:
from airo_camera_toolkit.image_transforms.transforms.crop import Crop

x_middle = observation.depth_map.shape[1] // 2
width = 350

x = x_middle - width // 2
y = 240
height = 600

crop_depth = Crop(observation.depth_map.shape, x=x, y=y, w=width, h=height)
crop_rgb_left = Crop(observation.image_left.shape, x=x, y=y, w=width, h=height)

depth_cropped = crop_depth.transform_image(observation.depth_map)

segmentation_mask = observation.depth_map < distance_max
segmentation_mask_cropped = crop_depth.transform_image(segmentation_mask)

crop_rgb_left = Crop(observation.image_left.shape, x=x, y=y, w=width, h=height)
image_left_cropped = crop_rgb_left.transform_image(observation.image_left)

depth_map_cropped_segmented = depth_cropped * segmentation_mask_cropped
image_left_cropped_segmented = image_left_cropped * segmentation_mask_cropped[:, :, None]


distance_cloth_min = np.min(depth_map_cropped_segmented[depth_map_cropped_segmented > 0.0])
distance_cloth_max = np.max(depth_map_cropped_segmented)

print("Shape of crop:", depth_cropped.shape)
print(f"Cloth distance range: {distance_cloth_min:.2f} - {distance_cloth_max:.2f}")

plt.figure(figsize=(18, 5))
plt.subplot(1, 5, 1)
plt.title("Cropped depth map")
plt.imshow(depth_cropped)
plt.subplot(1, 5, 2)
plt.title("Cropped segmentation mask")
plt.imshow(segmentation_mask_cropped)
plt.subplot(1, 5, 3)
plt.title("Segmented cropped depth map")
depth_map_im = plt.imshow(depth_map_cropped_segmented)
depth_map_im.set_clim(vmin=distance_cloth_min, vmax=distance_cloth_max) # increases contrast
plt.subplot(1, 5, 4)
plt.title("Cropped left image")
plt.imshow(image_left_cropped)
plt.subplot(1, 5, 5)
plt.title("Segmented cropped left image")
plt.imshow(image_left_cropped_segmented)
plt.show()


In [ ]:

rgb_left_cropped = crop_rgb_left.transform_image(observation.image_left)

In [ ]:
from cloth_tools.annotation.grasp_annotation import GraspAnnotation


grasp_dir = dataset_dir / "sample_000000" / "grasp"
grasp_annotation_file = grasp_dir / "grasp_annotation.json"

with open(grasp_annotation_file, "r") as f:
    grasp_annotation = GraspAnnotation.model_validate_json(f.read())


grasp_annotation

In [ ]:
image = observation.image_left

clicked_point = grasp_annotation.clicked_point_frontal
x, y = clicked_point

plt.figure(figsize=(12, 6))
plt.imshow(image)
plt.scatter(x, y, c="lawngreen", s=50, marker="x")
plt.show()

In [ ]:
image = image_left_cropped_segmented

clicked_point_in_crop = crop_rgb_left.transform_point(grasp_annotation.clicked_point_frontal)
x, y = clicked_point_in_crop

plt.figure(figsize=(12, 6))
plt.imshow(image)
plt.scatter(x, y, c="lawngreen", s=100, marker="x")
plt.show()